# Setup Enviroments 

### Download Request library

**NOTE**.  Only download this on initial start of platform

In [5]:
# Only execute on initial start of platform
# !pip install requests

### Define Spotify Developer Credentials

In [6]:
client_id = "cd1845f23d914228b14f6bed139ee594"
client_secret = ""

# Create an API Client Class for Spotify, `SpotifyAPI`

Rather than creating a function for Spotify Access Token that's a state-like item (change overtime), let's create a Class that can response to changes overtime.

This class authorize your access to Spotify API by: 
  1. Gain credentials by passing in Spotify Dev. ID and Secret Key
  1. Gain authorization by getting token
  

_NOTE_
    - Request Authorization's token uses "Basic" within the authorization header string.
    - While using the access token to access Spotify Web API uses "Bearer" within the authorization header string.
    - REF. https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow

In [7]:
import requests
import base64
import datetime
from urllib.parse import urlencode

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    
    token_URL = "https://accounts.spotify.com/api/token"
    
    
    
    
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    
    
    
    
    def get_client_credentials(self):
        """
        METHOD: Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if (client_id == None) or (client_secret == None):
            raise Exception("You must set a Client ID or a Client Secret with Spotify Developer Account.")
        
        # Convert Spotify Credentials from string to base64-bytes (decoded)
        client_creds = f"{client_id}:{client_secret}"                # type: String
        client_creds_b64 = base64.b64encode(client_creds.encode())   # type: Base64-Byte
        return client_creds_b64.decode()

    
    
    
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"             # Basic <base64 encoded client_id:client_secret>
        }
        
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
        
    
    
    
    
    def perform_auth(self):
        """
        METHOD: Authenticate to Spotify by acquiring token
        """
        token_URL = self.token_URL
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        # Get Token Response
        response = requests.post(token_URL, data=token_data, headers=token_headers)

        
        # Exit if Request is Invalid...
        if response.status_code not in range (200, 299):
            raise Exception("Could not authenticate client.")
            # return False        
        
        
        # Else, proceed...
        token_response_data = response.json()

        access_token = token_response_data['access_token']

        expires_in = token_response_data['expires_in']            # Unit: Seconds
        now = datetime.datetime.now()
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now

        return True
    
    
    
    
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    
    
    
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    
    
    
    
    def get_resource(self, lookup_id, resource_type="artists", sub_resource_type="", version="v1"):
        headers = self.get_resource_header()
        
        if sub_resource_type != "":
            endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}/{sub_resource_type}"
        else:
            endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        
        print(endpoint)
        
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        
        return r.json()
    
    
    
    
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type="artists")
    
    def get_artist_albums(self, _id):
        return self.get_resource(_id, resource_type="artists", sub_resource_type="albums")
    
    
    
    
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type="albums")
    
    def get_album_tracks(self, _id):
        return self.get_resource(_id, resource_type="albums", sub_resource_type="tracks")
    
    
    
    
    
    def get_track(self, _id):
        return self.get_resource(_id, resource_type="tracks")
        
    def get_track_features(self, _id):
        return self.get_resource(_id, resource_type="audio-features")

    
    
    
        
    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        print(lookup_url)
        
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}

        return r.json()
    
    
    
    
    
    def search(self, query=None, operator=None, operator_query=None, search_type="artist"):
        if query == None:
            raise Exception("A query is required")

        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        
        if (operator != None) and (operator_query != None):
            if (operator.lower() == "or") or (operator.lower() == "not"):
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

# Calling API Client Class, `SpotifyAPI`

In [8]:
sp = SpotifyAPI(client_id, client_secret)

# Search Spotify

The `Search` section of Spotify API endpoint retrieves Spotify Catalog information about albums, artists, playlists, tracks, shows or episodes that match a keyword string.

**REF**: https://developer.spotify.com/documentation/web-api/reference/search/search/

In [ ]:
# sp.search("Something Holy", search_type="Track")
sp.search(
    {
        "track": "Satellite",
        "artist": "Dave Matthews"
    },
    search_type="track"
)

In [ ]:
sp.search(query="Danger", operator="NOT", operator_query="Zone", search_type="track")

In [ ]:
sp.get_album("4r8HtFE8TGFtqEpMV7mU60")

In [10]:
import pandas as pd
from pandas import json_normalize

# 1.) Find Artists

In [11]:
# sp.search(query="Chainsmokers", search_type="artist")
artists = sp.search(query="Chainsmokers")

df_artists = json_normalize(artists)
# df_artists

q=Chainsmokers&type=artist
https://api.spotify.com/v1/search?q=Chainsmokers&type=artist


In [12]:
# df_artists['artists.items'][0]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/69GGBxA162lTqCwzJG5jLp'},
  'followers': {'href': None, 'total': 16851984},
  'genres': ['dance pop', 'edm', 'electropop', 'pop', 'tropical house'],
  'href': 'https://api.spotify.com/v1/artists/69GGBxA162lTqCwzJG5jLp',
  'id': '69GGBxA162lTqCwzJG5jLp',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/960547a625bc2eb742bb3dd170cbc049d2e94cf9',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/9da714082fe9696529abadc8e4095451221b4483',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/caca64268346846a0753ca894b6ff92bb4dfb864',
    'width': 160}],
  'name': 'The Chainsmokers',
  'popularity': 86,
  'type': 'artist',
  'uri': 'spotify:artist:69GGBxA162lTqCwzJG5jLp'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/5N4BnbcPsYktt2bD0CbKYq'},
  'followers': {'href': None, 'total': 1191},
  'genres': [],
  'href': 'https://api.spotify.com/v1/artists/5

In [7]:
# df_artists['artists.items'][0][0]

In [13]:
for i in range(df_artists['artists.total'][0]):
    artist_id = df_artists['artists.items'][0][i]['id']
    artist_name = df_artists['artists.items'][0][i]['name']
    print(f"ID:{artist_id}, Artist:{artist_name}")

ID:69GGBxA162lTqCwzJG5jLp, Artist:The Chainsmokers
ID:5N4BnbcPsYktt2bD0CbKYq, Artist:Chainsmokers


# 2.) List Artist's Albums

In [9]:
# List artist's albums, "The Chainsmokers", via Spotify Artist's ID, "69GGBxA162lTqCwzJG5jLp"
albums = sp.get_artist_albums("69GGBxA162lTqCwzJG5jLp")

df_albums = json_normalize(albums)
# df_albums

https://api.spotify.com/v1/artists/69GGBxA162lTqCwzJG5jLp/albums


In [10]:
# df_albums['items'][0][0]

In [11]:
for i in range(df_albums['total'][0]):
    album_id = df_albums['items'][0][i]['id']
    album_total_tracks = df_albums['items'][0][i]['total_tracks']
    album_name = df_albums['items'][0][i]['name']
    album_release_date = df_albums['items'][0][i]['release_date']
    print(f"Album ID:{album_id}, Number of Tracks:{album_total_tracks}, Album Name:{album_name}, Release Date:{album_release_date}")

Album ID:3cbcpRK1paF6lAxd16cmIE, Number of Tracks:15, Album Name:World War Joy (Japan Edition), Release Date:2020-02-19
Album ID:01GR4NL5O5CZM51k0aejKD, Number of Tracks:10, Album Name:World War Joy, Release Date:2019-12-06
Album ID:1AVNEke0ElkA1cekT0a1vO, Number of Tracks:23, Album Name:Sick Boy (Special Edition), Release Date:2019-07-10
Album ID:6ZvDJs17O3woQirttKRYCG, Number of Tracks:10, Album Name:Sick Boy, Release Date:2018-12-14
Album ID:4JPguzRps3kuWDD5GS6oXr, Number of Tracks:12, Album Name:Memories...Do Not Open, Release Date:2017-04-07
Album ID:3ShQFl9FladFKlonwPGZFc, Number of Tracks:7, Album Name:The Chainsmokers- Japan Special Edition, Release Date:2016-08-10
Album ID:4iheJNVeRsjwypaKGNTbzJ, Number of Tracks:1, Album Name:Closer (Tokyo Remix), Release Date:2020-02-06
Album ID:0KHCJzK0qQtlpz4B5oit6C, Number of Tracks:2, Album Name:Family - The Remixes, Release Date:2019-12-05
Album ID:0Yf5xlkmcqWPjBHeKx0rKc, Number of Tracks:3, Album Name:Push My Luck - The Remixes, Releas

IndexError: list index out of range

# 3.) List Artist-Album's Tracks

In [12]:
# List tracks for album, "Memories...Do Not Open", from The Chainsmokers via Spotify's Album ID, 4JPguzRps3kuWDD5GS6oXr"
tracks = sp.get_album_tracks("4JPguzRps3kuWDD5GS6oXr")

df_tracks = json_normalize(tracks)
# df_tracks

https://api.spotify.com/v1/albums/4JPguzRps3kuWDD5GS6oXr/tracks


In [13]:
# df_tracks['items'][0][0]

In [14]:
for i in range(df_tracks['total'][0]):
#     artist_name = df_tracks['items'][0][i]['artists'][0]['name']
#     artist_id = df_tracks['items'][0][i]['artists'][0]['id']
    track_id = df_tracks['items'][0][i]['id']
    track_num = df_tracks['items'][0][i]['track_number']
    track_name = df_tracks['items'][0][i]['name']
    print(f"Track ID:{track_id}, Track #:{track_num}, Track Name:{track_name}")

Track ID:0wfbD5rAksdXUzRvMfM3x5, Track #:1, Track Name:The One
Track ID:6cPyTS0Kk2sc4xQwC93kOg, Track #:2, Track Name:Break Up Every Night
Track ID:26AuyrZGzWWiYZPSd3XBIg, Track #:3, Track Name:Bloodstream
Track ID:1pJQAHpD51J7GYaFrrFO9S, Track #:4, Track Name:Don't Say (feat. Emily Warren)
Track ID:6RUKPb4LETWmmr3iAEQktW, Track #:5, Track Name:Something Just Like This
Track ID:6V9kwssTrwkKT72imgowj9, Track #:6, Track Name:My Type (feat. Emily Warren)
Track ID:5xhJmd0I15jFcEdqxfCzKk, Track #:7, Track Name:It Won't Kill Ya (feat. Louane)
Track ID:72jbDTw1piOOj770jWNeaG, Track #:8, Track Name:Paris
Track ID:6KjbNLbRjuoa8rEq5yNA6H, Track #:9, Track Name:Honest
Track ID:0dj1CtyRxZ4bnIT4Q20jNT, Track #:10, Track Name:Wake Up Alone (feat. Jhené Aiko)
Track ID:1KeZgPUr54C8iz3FjqzVoz, Track #:11, Track Name:Young
Track ID:6GeD5g9vZTz25Egf8kxoIY, Track #:12, Track Name:Last Day Alive (feat. Florida Georgia Line)


# 4.) List Tracks' Audio Features

In [15]:
# List track(s) audio features via track's Spotify ID
tracks = "0wfbD5rAksdXUzRvMfM3x5,6cPyTS0Kk2sc4xQwC93kOg,26AuyrZGzWWiYZPSd3XBIg"
request_string = f"?ids={tracks}"

df_tracks_audiofeatures = sp.get_track_features(request_string)

https://api.spotify.com/v1/audio-features/?ids=0wfbD5rAksdXUzRvMfM3x5,6cPyTS0Kk2sc4xQwC93kOg,26AuyrZGzWWiYZPSd3XBIg


In [26]:
# df_tracks_audiofeatures
df_tracks_audiofeatures['audio_features']

[{'danceability': 0.76,
  'energy': 0.303,
  'key': 2,
  'loudness': -11.362,
  'mode': 1,
  'speechiness': 0.0284,
  'acousticness': 0.238,
  'instrumentalness': 8.09e-06,
  'liveness': 0.294,
  'valence': 0.15,
  'tempo': 99.991,
  'type': 'audio_features',
  'id': '0wfbD5rAksdXUzRvMfM3x5',
  'uri': 'spotify:track:0wfbD5rAksdXUzRvMfM3x5',
  'track_href': 'https://api.spotify.com/v1/tracks/0wfbD5rAksdXUzRvMfM3x5',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0wfbD5rAksdXUzRvMfM3x5',
  'duration_ms': 177573,
  'time_signature': 4},
 {'danceability': 0.624,
  'energy': 0.806,
  'key': 1,
  'loudness': -5.957,
  'mode': 1,
  'speechiness': 0.0437,
  'acousticness': 0.00411,
  'instrumentalness': 0,
  'liveness': 0.0872,
  'valence': 0.528,
  'tempo': 149.999,
  'type': 'audio_features',
  'id': '6cPyTS0Kk2sc4xQwC93kOg',
  'uri': 'spotify:track:6cPyTS0Kk2sc4xQwC93kOg',
  'track_href': 'https://api.spotify.com/v1/tracks/6cPyTS0Kk2sc4xQwC93kOg',
  'analysis_url': 'https://ap

In [31]:
for i in range(len(df_tracks_audiofeatures['audio_features'])):
    track_id = df_tracks_audiofeatures['audio_features'][i]['id']
    track_danceability = df_tracks_audiofeatures['audio_features'][i]['danceability']
    track_energy = df_tracks_audiofeatures['audio_features'][i]['energy']
    track_loudness = df_tracks_audiofeatures['audio_features'][i]['loudness']
    track_speechiness = df_tracks_audiofeatures['audio_features'][i]['speechiness']
    track_acousticness = df_tracks_audiofeatures['audio_features'][i]['acousticness']
    track_instrumentalness = df_tracks_audiofeatures['audio_features'][i]['instrumentalness']
    track_liveness = df_tracks_audiofeatures['audio_features'][i]['liveness']
    track_valence = df_tracks_audiofeatures['audio_features'][i]['valence']
    track_tempo = df_tracks_audiofeatures['audio_features'][i]['tempo']
    print(
        f"Track ID:{track_id}, "
        f"danceability:{track_danceability}, "
        f"energy:{track_energy}, "
        f"loudness:{track_loudness}, "
        f"speechiness:{track_speechiness}, "
        f"acousticness:{track_acousticness}, "
        f"instrumentalness:{track_instrumentalness}, "
        f"liveness:{track_liveness}, "
        f"valence:{track_valence}, "
        f"tempo:{track_tempo}")

Track ID:0wfbD5rAksdXUzRvMfM3x5, danceability:0.76, energy:0.303, loudness:-11.362, speechiness:0.0284, acousticness:0.238, instrumentalness:8.09e-06, liveness:0.294, valence:0.15, tempo:99.991
Track ID:6cPyTS0Kk2sc4xQwC93kOg, danceability:0.624, energy:0.806, loudness:-5.957, speechiness:0.0437, acousticness:0.00411, instrumentalness:0, liveness:0.0872, valence:0.528, tempo:149.999
Track ID:26AuyrZGzWWiYZPSd3XBIg, danceability:0.62, energy:0.627, loudness:-5.889, speechiness:0.0259, acousticness:0.0317, instrumentalness:0, liveness:0.172, valence:0.164, tempo:90.955


# REFERENCES
  - Spotify User's Credentials https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow
  - Spotify Web API - Search https://developer.spotify.com/documentation/web-api/reference/search/search/
  - Spotify Web API - Artists https://developer.spotify.com/documentation/web-api/reference/artists/
  - Spotify Web API - Albums https://developer.spotify.com/documentation/web-api/reference/albums/